# HyperParameter Tuning

### `keras.wrappers.scikit_learn`

Example adapted from: [https://github.com/fchollet/keras/blob/master/examples/mnist_sklearn_wrapper.py]()

## Problem: 
Builds simple CNN models on MNIST and uses sklearn's GridSearchCV to find best model

In [1]:
import numpy as np
np.random.seed(1337)  # for reproducibility

In [2]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K

Using TensorFlow backend.


In [3]:
from sklearn.model_selection import GridSearchCV

# Data Preparation

In [4]:
nb_classes = 10

# input image dimensions
img_rows, img_cols = 28, 28

In [5]:
# load training data and do basic data normalization
(X_train, y_train), (X_test, y_test) = mnist.load_data()

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

11493376/11490434 [==============================] - 3s 0us/step


In [6]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

## Build Model

In [7]:
def make_model(dense_layer_sizes, filters, kernel_size, pool_size):
    '''Creates model comprised of 2 convolutional layers followed by dense layers

    dense_layer_sizes: List of layer sizes. This list has one number for each layer
    nb_filters: Number of convolutional filters in each convolutional layer
    nb_conv: Convolutional kernel size
    nb_pool: Size of pooling area for max pooling
    '''

    model = Sequential()

    model.add(Conv2D(filters, (kernel_size, kernel_size),
                     padding='valid', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(filters, (kernel_size, kernel_size)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size))
        model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    return model

In [8]:
dense_size_candidates = [[32], [64], [32, 32], [64, 64]]
my_classifier = KerasClassifier(make_model, batch_size=32)

## GridSearch HyperParameters

In [9]:
validator = GridSearchCV(my_classifier,
                         param_grid={'dense_layer_sizes': dense_size_candidates,
                                     # nb_epoch is avail for tuning even when not
                                     # an argument to model building function
                                     'epochs': [3, 6],
                                     'filters': [8],
                                     'kernel_size': [3],
                                     'pool_size': [2]},
                         scoring='neg_log_loss',
                         n_jobs=1)
validator.fit(X_train, y_train)

/Users/valerio/anaconda3/envs/dl-keras-tf/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/3
40000/40000 [==============================] - 20s 500us/step - loss: 0.6223 - acc: 0.7957
Epoch 2/3
40000/40000 [==============================] - 20s 493us/step - loss: 0.3574 - acc: 0.8888
Epoch 3/3
40000/40000 [==============================] - 22s 557us/step - loss: 0.2970 - acc: 0.9080
Epoch 1/3
40000/40000 [==============================] - 20s 491us/step - loss: 0.6444 - acc: 0.7862
Epoch 2/3
40000/40000 [==============================] - 21s 534us/step - loss: 0.3627 - acc: 0.8850
Epoch 3/3
40000/40000 [==============================] - 22s 541us/step - loss: 0.3117 - acc: 0.9000
Epoch 1/3
40000/40000 [==============================] - 23s 564us/step - loss: 0.6102 - acc: 0.8000
Epoch 2/3
40000/40000 [==============================] - 21s 514us/step - loss: 0.3535 - acc: 0.8902
Epoch 3/3
40000/40000 [==============================] - 22s 547us/step - loss: 0.2916 - acc: 0.9081
Epoch 1/6
40000/40000 [==============================] - 26s 658us/step - loss: 0.6897 - ac

40000/40000 [==============================] - 24s 589us/step - loss: 0.3671 - acc: 0.8882
Epoch 2/3
40000/40000 [==============================] - 22s 543us/step - loss: 0.1551 - acc: 0.9580
Epoch 3/3
40000/40000 [==============================] - 20s 495us/step - loss: 0.1209 - acc: 0.9670
Epoch 1/3
40000/40000 [==============================] - 20s 501us/step - loss: 0.3823 - acc: 0.8829
Epoch 2/3
40000/40000 [==============================] - 19s 478us/step - loss: 0.1637 - acc: 0.9539
Epoch 3/3
40000/40000 [==============================] - 19s 468us/step - loss: 0.1253 - acc: 0.9663
Epoch 1/3
40000/40000 [==============================] - 21s 524us/step - loss: 0.3987 - acc: 0.8782
Epoch 2/3
40000/40000 [==============================] - 20s 499us/step - loss: 0.1716 - acc: 0.9535
Epoch 3/3
40000/40000 [==============================] - 20s 509us/step - loss: 0.1289 - acc: 0.9657
Epoch 1/6
40000/40000 [==============================] - 22s 549us/step - loss: 0.3834 - acc: 0.8851


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x11ea29828>,
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'dense_layer_sizes': [[32], [64], [32, 32], [64, 64]], 'epochs': [3, 6], 'filters': [8], 'kernel_size': [3], 'pool_size': [2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_log_loss', verbose=0)

In [10]:
print('The parameters of the best model are: ')
print(validator.best_params_)

# validator.best_estimator_ returns sklearn-wrapped version of best model.
# validator.best_estimator_.model returns the (unwrapped) keras model
best_model = validator.best_estimator_.model
metric_names = best_model.metrics_names
metric_values = best_model.evaluate(X_test, y_test)
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)

The parameters of the best model are: 
{'dense_layer_sizes': [64, 64], 'epochs': 6, 'filters': 8, 'kernel_size': 3, 'pool_size': 2}
10000/10000 [==============================] - 2s 249us/step
loss :  0.04074383126419634
acc :  0.989


---

# There's more:

The `GridSearchCV` model in scikit-learn performs a complete search, considering **all** the possible combinations of Hyper-parameters we want to optimise.

If we want to apply for an optmised and bounded search in the hyper-parameter space, I strongly suggest to take a look at:

* `Keras + hyperopt == hyperas`: [http://maxpumperla.github.io/hyperas/](http://maxpumperla.github.io/hyperas/)